In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [2]:
#initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API key exists and begins: {openai_api_key[:8]}")
else:
    print("OpenAI api key not set")
MODEL = "gpt-4.1-mini"
openai = OpenAI()


OpenAI API key exists and begins: sk-proj-


In [3]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [4]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [5]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [6]:
get_ticket_price("Paris")

DATABASE TOOL CALLED: Getting price for Paris


'Ticket price to Paris is $899.0'

In [7]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [8]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [9]:
get_ticket_price("Tokyo")

DATABASE TOOL CALLED: Getting price for Tokyo


'Ticket price to Tokyo is $1420.0'

In [10]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

book_flight_function = {
    "name": "book_flight",
    "description": "Book a return flight for a passenger.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "City the passenger is traveling to"
            },
            "name": {
                "type": "string",
                "description": "Full name of the passenger"
            },
            "date": {
                "type": "string",
                "description": "Travel date in YYYY-MM-DD format"
            }
        },
        "required": ["destination_city", "passenger_name", "date"],
        "additionalProperties": False
    }
}


In [11]:
def initialize_database():
    with sqlite3.connect(DB) as conn:
        cursor=conn.cursor()
        cursor.execute(
        """CREATE TABLE IF NOT EXISTS bookings(id INTEGER PRIMARY KEY AUTOINCREMENT,
           name TEXT, city TEXT, date TEXT
    )""")
conn.commit()

In [12]:
initialize_database()

In [13]:
tools = [{"type": "function", "function": price_function},{"type":"function","function":book_flight_function}]

In [14]:
def book_flight(destination_city,name,date):
    print(f"BOOKING TOOL CALLED: {name} -> {destination_city} on {date}",flush=True)
    with sqlite3.connect(DB) as conn:
        cursor=conn.cursor()
        cursor.execute("""
        INSERT INTO bookings(name,city,date)
        VALUES (?,?,?)
        """, (name,destination_city.lower(),date))
        booking_id=cursor.lastrowid
        conn.commit()
    return f"Flight booked! Your booking ID is {booking_id}"

In [15]:
def chat(message,history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message=response.choices[0].message
        responses=handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response=openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
    return response.choices[0].message.content

In [16]:
def handle_tool_calls(message):
    responses=[]
    for tool_call in message.tool_calls:
        arguments=json.loads(tool_call.function.arguments)
        if tool_call.function.name == "get_ticket_price":
            city = arguments.get('destination_city')
            result = get_ticket_price(city)
        elif tool_call.function.name == "book_flight":
            city = arguments.get('destination_city')
            name = arguments.get('name')
            date = arguments.get('date')
            result = book_flight(city,name,date)
        else:
            result = "Unknown tool."
    responses.append({
        "role":"tool",
        "content":result,
        "tool_call_id":tool_call.id
        })
    return responses

In [17]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


BOOKING TOOL CALLED: kendall roy -> paris on 2026-02-19
